<a href="https://colab.research.google.com/github/EdBerg21/AI-Based-Fraud-Detection/blob/main/one_shot_talking_face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 torchtext==0.14.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu116 -U
!git lfs install
%cd /content
!git clone https://huggingface.co/camenduru/pocketsphinx-20.04-t4 pocketsphinx
%cd  /content/pocketsphinx
!sudo cmake --build build --target install
%cd /content
!git clone https://huggingface.co/camenduru/one-shot-talking-face-20.04-t4 one-shot-talking-face
%cd /content/one-shot-talking-face
!pip install -r /content/one-shot-talking-face/requirements.txt
!chmod 755 /content/one-shot-talking-face/OpenFace/FeatureExtraction
!mkdir /content/out
!apt install -qq libgtk2.0-0 jq -y
!pip install -q imageio-ffmpeg numpy==1.23.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 GB 723.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 77.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 70.4 MB/s eta 0:00:00
Git LFS initialized.
/content
Cloning into 'pocketsphinx'...
remote: Enumerating objects: 1658, done.
remote: Total 1658 (delta 0), reused 0 (delta 0), pack-reused 1658 (from 1)
Receiving objects: 100% (1658/1658), 8.51 MiB | 19.63 MiB/s, done.
Resolving deltas: 100% (592/592), done.
/content/pocketsphinx
-- The C compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- SIZEOF_LONG="8" ; SIZEOF_LONG_LONG="8"
-- WORDS_BI

In [2]:
!wget https://github.com/camenduru/one-shot-talking-face-colab/raw/main/test/audio.wav -O /content/audio.wav
!wget https://github.com/camenduru/one-shot-talking-face-colab/blob/main/test/image.png?raw=true -O /content/image.png

--2024-12-08 17:57:08--  https://github.com/camenduru/one-shot-talking-face-colab/raw/main/test/audio.wav
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/camenduru/one-shot-talking-face-colab/main/test/audio.wav [following]
--2024-12-08 17:57:09--  https://raw.githubusercontent.com/camenduru/one-shot-talking-face-colab/main/test/audio.wav
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 608334 (594K) [audio/wav]
Saving to: ‘/content/audio.wav’

/content/audio.wav  100%[===================>] 594.08K  --.-KB/s    in 0.03s   

2024-12-08 17:57:09 (18.1 MB/s) - ‘/content/audio.wav’ saved [608334/6083

In [5]:
import os, random, torchaudio
from IPython.display import HTML
from base64 import b64encode
from IPython.display import clear_output

def show_video(video_path, video_width = 256):
  video_file = open(video_path, "r+b").read()
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")

wav_file = "/content/audio.wav"  #@param {type:"string"}
image_file = "/content/image.png" #@param {type:"string"}

waveform, sample_rate = torchaudio.load(wav_file)
torchaudio.save(wav_file, waveform, sample_rate, encoding="PCM_S", bits_per_sample=16)

os.environ['wav_file'] = wav_file
os.environ['image_file'] = image_file
random_int = str(random.randint(1, 1000000))

!mkdir /content/train
!cp $wav_file /content/train/audio.wav
!cp $image_file /content/train/image.png

!pocketsphinx -phone_align yes single /content/train/audio.wav $text | jq '[.w[]|{word: (.t | ascii_upcase | sub("<S>"; "sil") | sub("<SIL>"; "sil") | sub("\\(2\\)"; "") | sub("\\(3\\)"; "") | sub("\\(4\\)"; "") | sub("\\[SPEECH\\]"; "SIL") | sub("\\[NOISE\\]"; "SIL")), phones: [.w[]|{ph: .t | sub("\\+SPN\\+"; "SIL") | sub("\\+NSN\\+"; "SIL"), bg: (.b*100)|floor, ed: (.b*100+.d*100)|floor}]}]' > /content/test.json
%cd /content/one-shot-talking-face
!python -B test_script.py --img_path /content/train/image.png --audio_path /content/train/audio.wav --phoneme_path /content/test.json --save_dir /content/train

os.environ['ran_num'] = random_int
!cp /content/train/image_audio.mp4 "/content/out/${ran_num}.mp4"

clear_output()

show_video(f"/content/out/{random_int}.mp4")